# import lib and test GPU of tensorflow


In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
# import matplotlib.image as mping
import matplotlib.animation as animation
import time, datetime
import sys
import gc
import os
import tensorflow as tf
import h5py
from random import randint
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import manifold, datasets
from IPython.display import clear_output
from tsnecuda import TSNE
#import tsnecuda
#tsnecuda.test()


In [28]:
# os.environ["QT_API"] = "pyqt5"
# print(os.environ.get('QT_API'))
import tsnecuda
print(tsnecuda.__file__)


/home/shawn/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tsnecuda/__init__.py


In [3]:
%matplotlib qt


In [4]:
#test tesnsorflow
print("tensorflow version: ",tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
# Returns whether TensorFlow was built with CUDA (GPU) support.
print("tf.test.is_Built_With_cuda = ",tf.test.is_built_with_cuda())
# RetUrns whether TensorFlow can access a GPU. (deprecated)
print("tf.test.is_gpu_available = ",tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))


tensorflow version:  2.1.0
Num GPUs Available:  1
Num CPUs Available:  1
tf.test.is_Built_With_cuda =  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


tf.test.is_gpu_available =  True


# Main program -------------->
# read data(.npy)


In [5]:
def read_data_and_build_data_label():
    folder_path = '/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/spectral_curve_v1/A/*' # Shawn Linux

    file_name = sorted(glob.glob(folder_path))
    # print('\n'.join(file_name))
    data_n = len(file_name)
    # print(data_n)

    ## 0511 --> 0
    ## 0601 --> 8
    label_value = 0
    data_label = np.zeros(data_n, dtype=np.int8)
    check = file_name[0][-16:-12]
    for i in range(data_n):
        if file_name[i][-16:-12] != check:
            label_value += 1
            check = file_name[i][-16:-12]
        data_label[i] = label_value

    ## check label
    # for i in range(data_n):
    #     print("{}\t{}".format(data_label[i], file_name[i][-16:-12]))

    data = np.zeros((data_n,300))
    for i in range(data_n):
        data[i] = np.load(file_name[i])

    return data_n, data_label, data

data_n, data_label, data = read_data_and_build_data_label()


In [17]:
def show_spectral_curve(data_label, data):
    colormap = plt.cm.rainbow(np.linspace(0, 1, data_label[-1]+1))

    plt.ion()
    plt.xlim(0,300)
    plt.ylim(0,1.5)
    plt.xticks(np.arange(0, 300, 50))

    for i in range(data_n):
        plt.plot(np.arange(0,300,1),data[i],color=colormap[data_label[i]])
        plt.pause(0.001)

    plt.show()

show_spectral_curve(data_label, data)

# t-SNE


In [84]:
n_components = 2
perplexity = 11
early_exaggeration = 20
learning_rate = 150
n_iter = 2000
n_iter_without_progress = 300
min_grad_norm = 1e-3
metric = 'euclidean'
init = 'random'
verbose = 1

range_ = 1
start = 400

data_embedded = TSNE(perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter, verbose=1).fit_transform(data)

# show data
colormap = plt.cm.rainbow(np.linspace(0, 1, data_label[-1]+1))

plt.clf()
plt.subplot(1,1,1)
for i in range(data_n):
    plt.scatter(data_embedded[i][0], data_embedded[i][1], color=colormap[data_label[i]])
plt.show

<function matplotlib.pyplot.show(*args, **kw)>

# test


In [ ]:
print(data[1].shape)
print(data.shape)
plt.plot(np.arange(0,300,dtype=np.int16), data[3])